<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Cleaning in Python

The data we have collected may have several issues we need to identify:

* Are there missing values? How are they represented?
* Is the format of the table ready to be analyzed? Are there other elements not relevant but distracting or likely to confuse our work?
* Is every cell well written? are there characters that may not allow future analysis?

Let's check some data:

In [1]:
import pandas as pd

wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices" 
freedomDFs=pd.read_html(wikiLink, flavor='bs4',attrs={'class':'wikitable sortable'})
len(freedomDFs)

2

Let's keep the first one:

In [2]:
freedom=freedomDFs[0].copy()
freedom.head()

,Country,Freedom in the World 2023[13],Score,Index of Economic Freedom 2023[14],Score.1,Press Freedom Index 2023[3],Score.2,Democracy Index 2023[9],Score.3
0,Norway,free,100,mostly free,76.9,good,95.18,full democracy,9.81
1,Ireland,free,97,free,82,good,89.91,full democracy,9.05
2,Sweden,free,100,mostly free,77.5,good,88.15,full democracy,9.26
3,Finland,free,100,mostly free,77.1,good,87.94,full democracy,9.2
4,Denmark,free,97,mostly free,77.6,good,89.48,full democracy,9.15


## 1. Clean headers

In [3]:
# check headers
freedom.columns

Index(['Country', 'Freedom in the World 2023[13]', 'Score',
       'Index of Economic Freedom 2023[14]', 'Score.1',
       'Press Freedom Index 2023[3]', 'Score.2', 'Democracy Index 2023[9]',
       'Score.3'],
      dtype='object')

Cleaning requires a strategy. In the strings above your main problem is the footnotes and the quasi-duplicates. 

In [4]:
# the quasi duplicates
ScoreColumns=freedom.columns[freedom.columns.str.contains('Scor')]
ScoreColumns

Index(['Score', 'Score.1', 'Score.2', 'Score.3'], dtype='object')

In [5]:
# the not quasi duplicates
freedom.columns[~freedom.columns.str.contains('Scor')]

Index(['Country', 'Freedom in the World 2023[13]',
       'Index of Economic Freedom 2023[14]', 'Press Freedom Index 2023[3]',
       'Democracy Index 2023[9]'],
      dtype='object')

In [6]:
# save the last one but the first one:
notScoreColumns=freedom.columns[~freedom.columns.str.contains('Scor')][1:]
notScoreColumns

Index(['Freedom in the World 2023[13]', 'Index of Economic Freedom 2023[14]',
       'Press Freedom Index 2023[3]', 'Democracy Index 2023[9]'],
      dtype='object')

Let's keep the last ones without the footnotes, let's _divide and conquer_ using **split()**:

In [7]:
# using list comprehension
[element.split('[') for element in notScoreColumns]

[['Freedom in the World 2023', '13]'],
 ['Index of Economic Freedom 2023', '14]'],
 ['Press Freedom Index 2023', '3]'],
 ['Democracy Index 2023', '9]']]

You see how I split each element, but the resulting list is not what you want, you need to keep the first element only:

In [8]:
# keeping first element [0]
[element.split('[')[0] for element in notScoreColumns]

['Freedom in the World 2023',
 'Index of Economic Freedom 2023',
 'Press Freedom Index 2023',
 'Democracy Index 2023']

This is not bad at all. However, a more efficient alternative is using **regular expressions**. There are books about this topic, but I will share some patterns that may prove useful.

In this situation, I want to:

* Get rid of footnotes.
* Get rid of the years.

Let's see:

In [9]:
import re  # a package to use regular expressions.

# one or more consecutive number \\d+
# anything that looks like \\[\\w+\\]
# using '|' as or
# using .strip() for unwanted spaces

pattern='\\d+|\\[\\w+\\]'
nothing=''

#substitute the 'pattern' by 'nothing':
[re.sub(pattern,nothing,element).strip() for element in notScoreColumns]

['Freedom in the World',
 'Index of Economic Freedom',
 'Press Freedom Index',
 'Democracy Index']

In [10]:
#save result
notScoreColumnsCleaner=[re.sub(pattern,nothing,element).strip() for element in notScoreColumns]

Let's create acronyms:

In [11]:
# split into list words
[nameCol.split() for nameCol in notScoreColumnsCleaner]

[['Freedom', 'in', 'the', 'World'],
 ['Index', 'of', 'Economic', 'Freedom'],
 ['Press', 'Freedom', 'Index'],
 ['Democracy', 'Index']]

In [12]:
# first letter of each word as list
[[letter[0] for letter in nameCol.split()] for nameCol in notScoreColumnsCleaner]

[['F', 'i', 't', 'W'], ['I', 'o', 'E', 'F'], ['P', 'F', 'I'], ['D', 'I']]

In [13]:
# concatenate first letters in each list
["".join([letter[0] for letter in nameCol.split()]) for nameCol in notScoreColumnsCleaner]

['FitW', 'IoEF', 'PFI', 'DI']

We saved the acronyms:

In [14]:
acronyms=["".join([letter[0] for letter in nameCol.split()]) for nameCol in notScoreColumnsCleaner]
acronyms

['FitW', 'IoEF', 'PFI', 'DI']

We concatenate "score" to the acronyms into another list:

In [15]:
acronyms_score=[acro+'_score' for acro in acronyms]
acronyms_score

['FitW_score', 'IoEF_score', 'PFI_score', 'DI_score']

Let's rename:

In [16]:
change={old:new for old,new in zip(ScoreColumns,acronyms_score)}
change

{'Score': 'FitW_score',
 'Score.1': 'IoEF_score',
 'Score.2': 'PFI_score',
 'Score.3': 'DI_score'}

In [17]:
change2={old:new for old,new in zip(notScoreColumns,acronyms)}
change2

{'Freedom in the World 2023[13]': 'FitW',
 'Index of Economic Freedom 2023[14]': 'IoEF',
 'Press Freedom Index 2023[3]': 'PFI',
 'Democracy Index 2023[9]': 'DI'}

In [18]:
change.update(change2)
change

{'Score': 'FitW_score',
 'Score.1': 'IoEF_score',
 'Score.2': 'PFI_score',
 'Score.3': 'DI_score',
 'Freedom in the World 2023[13]': 'FitW',
 'Index of Economic Freedom 2023[14]': 'IoEF',
 'Press Freedom Index 2023[3]': 'PFI',
 'Democracy Index 2023[9]': 'DI'}

In [19]:
freedom.rename(columns=change,inplace=True)
freedom.head()

,Country,FitW,FitW_score,IoEF,IoEF_score,PFI,PFI_score,DI,DI_score
0,Norway,free,100,mostly free,76.9,good,95.18,full democracy,9.81
1,Ireland,free,97,free,82,good,89.91,full democracy,9.05
2,Sweden,free,100,mostly free,77.5,good,88.15,full democracy,9.26
3,Finland,free,100,mostly free,77.1,good,87.94,full democracy,9.2
4,Denmark,free,97,mostly free,77.6,good,89.48,full democracy,9.15


## 2. Clean the data values.

Since there are categories, we could try some frequency tables:

In [20]:
freedom.FitW.value_counts(dropna=False).sort_index()

FitW
free           84
not free       57
partly free    55
NaN             1
Name: count, dtype: int64

In [21]:
freedom.IoEF.value_counts(dropna=False).sort_index()

IoEF
free                4
moderately free    56
mostly free        23
mostly unfree      65
repressed          28
NaN                21
Name: count, dtype: int64

In [22]:
freedom.PFI.value_counts(dropna=False).sort_index()

PFI
difficult       42
good             8
problematic     59
satisfactory    44
very serious    31
NaN             13
Name: count, dtype: int64

In [23]:
freedom.DI.value_counts(dropna=False).sort_index()

DI
authoritarian regime    57
flawed democracy        51
full democracy          23
hybrid regime           34
NaN                     32
Name: count, dtype: int64

The categories are well written.

Let's see the numeric columns. Let's identify cell that do not have numeric strings:


In [ ]:
set(freedom.FitW_score[~freedom.FitW_score.str.contains('^\\d+\\.*\\d*$')])

Then, we can generalize:

In [ ]:
badValues=[]
for col in acronyms_score:
    currentBad=freedom.loc[:,col][~freedom.loc[:,col].str.contains('^\\d+\\.*\\d*$')]
    badValues.extend(currentBad)

badValues=list(set(badValues))
badValues

We will need to replace those values with a proper missing value:

In [ ]:
import numpy as np

freedom.replace(to_replace=badValues, value=np.nan,inplace=True)

In [ ]:
freedom.info()

Let's keep the complete data:

In [ ]:
freedom.dropna(how='any',axis=0,inplace=True, # keep complete
               ignore_index=True) # reset index

freedom

## 3. Check key column

In [ ]:
freedom.Country

In [ ]:
# to upper case and no trailing or leading spaces
freedom.Country.str.upper().str.strip()

In [ ]:
freedom['Country']=freedom.Country.str.upper().str.strip()

## 4. Save output

In [ ]:
freedom.to_csv('freedom_Py.csv',index=False)